In [ ]:
pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 28.1 MB/s eta 0:00:00


In [ ]:
pip install pytorch-forecasting torch lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 6.4 MB/s eta 0:00:00


In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import yfinance as yf  # for fetching stock data
import matplotlib.pyplot as plt
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss, MAE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
l1 = []
for i in os.listdir('/content/drive/MyDrive/processed'):
  l1.append(i.strip("processed_news_sentiment_").strip(".csv"))

In [ ]:
with open('/content/all_tickers.txt', 'r') as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]
print(lines)

['ADBE', 'BX', 'MET', 'DLR', 'TJX', 'TT', 'BN', 'MA', 'CPRT', 'COST', 'NOC', 'UNP', 'BKNG', 'MDLZ', 'ADI', 'WM', 'TDG', 'CI', 'SYK', 'MU', 'ABBV', 'ABNB', 'PEP', 'NOW', 'LLY', 'ISRG', 'CSCO', 'EQIX', 'HCA', 'GILD', 'VRTX', 'APH', 'VZ', 'CL', 'MSI', 'FCX', 'CDNS', 'PSX', 'BAC', 'UAL', 'CHTR', 'MDT', 'BK', 'MS', 'ROP', 'CTAS', 'MSFT', 'ZTS', 'FDX', 'RGNX', 'AMAT', 'AON', 'AZO', 'AXP', 'UBER', 'CME', 'PH', 'AMP', 'CVX', 'XOM', 'ICE', 'PCG', 'NXPI', 'KLAC', 'NVDA', 'ACN', 'SCHW', 'SQ', 'CMG', 'SHW', 'SPG', 'AMD', 'O', 'URI', 'RTX', 'PNC', 'SBUX', 'ORCL', 'TXN', 'MMM', 'TMO', 'INTU', 'DHI', 'INTC', 'AMZN', 'MNST', 'UPS', 'NEE', 'ELV', 'EMR', 'ECL', 'WELL', 'AMT', 'GE', 'PCAR', 'MCD', 'PM', 'ADSK', 'CAT', 'GS', 'GD', 'FTNT', 'PYPL', 'BDX', 'KMI', 'FICO', 'D', 'DE', 'SRE', 'TSLA', 'RCL', 'ANET', 'ABT', 'CVS', 'META', 'SO', 'COF', 'PAYX', 'COP', 'DHR', 'MCK', 'CRWD', 'HD', 'ORLY', 'WFC', 'NEM', 'FANG', 'MCO', 'DIS', 'PG', 'ALL', 'T', 'C', 'ETN', 'GWW', 'NRG', 'NSC', 'SPGI', 'NKE', 'V', 'KO', '

In [ ]:
len(lines)

199

In [ ]:
# Code for getting tickers of the Top 200 companies by market cap
# Load the S&P 500 company tickers from Wikipedia
# url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
# sp500_table = pd.read_html(url)
# sp500_df = sp500_table[0]  # The first table on the page is usually the one we want
# tickers = sp500_df['Symbol'].tolist()  # Get the list of tickers

tickers = lines
# Function to get the market cap for each ticker
def get_market_cap(ticker):
    try:
        stock = yf.Ticker(ticker)
        market_cap = stock.info.get('marketCap')
        return market_cap
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

# Create a list to store the tickers and their market caps
ticker_data = []

for ticker in tickers:
    market_cap = get_market_cap(ticker)
    if market_cap is not None:
        ticker_data.append({'Ticker': ticker, 'MarketCap': market_cap})
    else:
        print(f"Skipping {ticker} due to error")

# Create a DataFrame and sort by market cap
ticker_df = pd.DataFrame(ticker_data)
ticker_df = ticker_df.sort_values(by='MarketCap', ascending=False)

# Select the top 200 companies by market cap
top_200 = ticker_df.head(200)
print(top_200['Ticker'].tolist())  # Print the list of top 200 tickers


['NVDA', 'AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'TSLA', 'AVGO', 'LLY', 'WMT', 'V', 'UNH', 'XOM', 'ORCL', 'MA', 'COST', 'HD', 'PG', 'NFLX', 'JNJ', 'BAC', 'CRM', 'ABBV', 'CVX', 'KO', 'TMUS', 'WFC', 'BX', 'CSCO', 'ACN', 'AMD', 'ADBE', 'PEP', 'MS', 'LIN', 'NOW', 'DIS', 'MCD', 'AXP', 'IBM', 'PM', 'ABT', 'TMO', 'ISRG', 'GE', 'INTU', 'CAT', 'GS', 'TXN', 'VZ', 'QCOM', 'BKNG', 'DHR', 'T', 'RTX', 'NEE', 'SPGI', 'AMGN', 'PGR', 'BLK', 'SYK', 'SCHW', 'UBER', 'ETN', 'UNP', 'AMAT', 'PFE', 'PLTR', 'TJX', 'BSX', 'PANW', 'C', 'COP', 'LMT', 'ANET', 'ADP', 'FI', 'DE', 'BMY', 'VRTX', 'CB', 'MU', 'SBUX', 'UPS', 'GILD', 'NKE', 'MDT', 'BA', 'ADI', 'PLD', 'INTC', 'DELL', 'SO', 'SHW', 'MO', 'AMT', 'LRCX', 'GEV', 'ELV', 'TT', 'CI', 'ICE', 'PH', 'CTAS', 'EQIX', 'WM', 'DUK', 'APH', 'CRWD', 'BN', 'MDLZ', 'MCO', 'WELL', 'SNPS', 'PYPL', 'KLAC', 'HCA', 'CDNS', 'ABNB', 'AON', 'MSI', 'REGN', 'CMG', 'CME', 'PNC', 'USB', 'MCK', 'ZTS', 'ITW', 'CEG', 'MAR', 'CL', 'GD', 'EOG', 'EMR', 'APD', 'WMB', 'NOC', 'FDX', 'FTNT', 'CVS

In [ ]:
def convert_data_type(df, time_cols=[], float_cols=[], cat_cols=[]):
    for col in time_cols:
        df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
    for col in float_cols:
        df[col] = df[col].astype(float)
    for col in cat_cols:
        df[col] = df[col].astype('category')
    return df

# Define the stock tickers and the date range
tickers = lines
start_date = "2022-11-01"
end_date = "2024-11-03"

In [ ]:
# Download historical data from yfinance
df = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

[*********************100%***********************]  199 of 199 completed


In [ ]:
df.head()

Ticker                            BLK                                      \
Price                            Open        High         Low       Close   
Date                                                                        
2022-11-01 00:00:00+00:00  656.739990  658.140015  648.510010  652.000000   
2022-11-02 00:00:00+00:00  650.489990  663.219971  637.900024  638.809998   
2022-11-03 00:00:00+00:00  626.440002  638.619995  622.979980  633.809998   
2022-11-04 00:00:00+00:00  646.070007  660.390015  642.679993  656.150024   
2022-11-07 00:00:00+00:00  660.250000  678.570007  655.549988  676.299988   

Ticker                                                 CME              \
Price                       Adj Close   Volume        Open        High   
Date                                                                     
2022-11-01 00:00:00+00:00  617.754517   771100  174.509995  175.300003   
2022-11-02 00:00:00+00:00  605.257385  1061500  173.220001  175.740005   
2022-11-03 00:00:00+00:00  600.519958   717200  171.130005  173.330002   
2022-11-04 00:00:00+00:00  621.686584   707300  171.889999  172.919998   
2022-11-07 00:00:00+00:00  640.778198   798500  171.139999  171.550003   

Ticker                                             ...        CTAS  \
Price                             Low       Close  ...         Low   
Date                                               ...               
2022-11-01 00:00:00+00:00  172.000000  173.529999  ...  106.035004   
2022-11-02 00:00:00+00:00  171.759995  172.880005  ...  103.922501   
2022-11-03 00:00:00+00:00  170.800003  170.919998  ...  102.547501   
2022-11-04 00:00:00+00:00  169.220001  170.669998  ...  101.407501   
2022-11-07 00:00:00+00:00  169.389999  169.740005  ...  102.802498   

Ticker                                                            AEP  \
Price                           Close   Adj Close   Volume       Open   
Date                                                                    
2022-11-01 00:00:00+00:00  107.067497  102.533691  1466000  88.099998   
2022-11-02 00:00:00+00:00  103.952499   99.550591  1627200  88.510002   
2022-11-03 00:00:00+00:00  102.735001   98.384659  1742800  87.769997   
2022-11-04 00:00:00+00:00  102.660004   98.312820  2230000  88.820000   
2022-11-07 00:00:00+00:00  104.964996  100.520218  2813200  89.839996   

Ticker                                                                          
Price                           High        Low      Close  Adj Close   Volume  
Date                                                                            
2022-11-01 00:00:00+00:00  89.199997  87.620003  88.529999  80.975853  4169800  
2022-11-02 00:00:00+00:00  90.639999  88.169998  88.209999  80.683167  3166600  
2022-11-03 00:00:00+00:00  89.580002  87.230003  88.680000  81.113060  4020800  
2022-11-04 00:00:00+00:00  89.680000  87.720001  89.570000  81.927109  3304300  
2022-11-07 00:00:00+00:00  89.839996  86.680000  88.000000  80.491089  3038600  

[5 rows x 1194 columns]

In [ ]:
df = df.stack(level=0).reset_index()
df.columns = ['time', 'ticker', 'open', 'high', 'low', 'close', 'adjclose', 'volume']
df = convert_data_type(df, time_cols=['time'], float_cols=['open', 'high', 'low', 'close', 'volume'], cat_cols=['ticker'])

# Set the time as index and sort values by time
df = df.set_index('time', drop=True).reset_index()
df = df.groupby('ticker').apply(lambda x: x.sort_values('time')).reset_index(drop=True)


<ipython-input-15-48021d768e11>:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=0).reset_index()
<ipython-input-15-48021d768e11>:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = df.groupby('ticker').apply(lambda x: x.sort_values('time')).reset_index(drop=True)
<ipython-input-15-48021d768e11>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly se

In [ ]:
df

,time,ticker,open,high,low,close,adjclose,volume
0,2022-11-01 00:00:00+00:00,AAPL,155.080002,155.449997,149.130005,150.649994,148.835358,80379300.0
1,2022-11-02 00:00:00+00:00,AAPL,148.949997,152.169998,145.000000,145.029999,143.283066,93604600.0
2,2022-11-03 00:00:00+00:00,AAPL,142.059998,142.800003,138.750000,138.880005,137.207169,97918500.0
3,2022-11-04 00:00:00+00:00,AAPL,142.089996,142.669998,134.380005,138.380005,136.939941,140814800.0
4,2022-11-07 00:00:00+00:00,AAPL,137.110001,139.149994,135.669998,138.919998,137.474319,83374600.0
...,...,...,...,...,...,...,...,...
99938,2024-10-28 00:00:00+00:00,ZTS,181.529999,183.100006,180.699997,182.759995,182.327942,1909700.0
99939,2024-10-29 00:00:00+00:00,ZTS,182.009995,184.410004,181.039993,181.270004,180.841476,1696000.0
99940,2024-10-30 00:00:00+00:00,ZTS,179.919998,183.809998,179.800003,182.740005,182.307999,1957800.0
99941,2024-10-31 00:00:00+00:00,ZTS,180.220001,181.580002,178.740005,178.779999,178.779999,2120000.0


In [ ]:
required_df = df[['ticker','time','close']]

,ticker,time,close
0,AAPL,2022-11-01 00:00:00+00:00,150.649994
1,AAPL,2022-11-02 00:00:00+00:00,145.029999
2,AAPL,2022-11-03 00:00:00+00:00,138.880005
3,AAPL,2022-11-04 00:00:00+00:00,138.380005
4,AAPL,2022-11-07 00:00:00+00:00,138.919998
...,...,...,...
99938,ZTS,2024-10-28 00:00:00+00:00,182.759995
99939,ZTS,2024-10-29 00:00:00+00:00,181.270004
99940,ZTS,2024-10-30 00:00:00+00:00,182.740005
99941,ZTS,2024-10-31 00:00:00+00:00,178.779999


In [ ]:
# import yfinance as yf
# import pandas as pd
# from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

# # Assuming 'lines' is your list of tickers
# tickers = lines

# # Function to get the market cap for each ticker
# def get_market_cap(ticker):
#     try:
#         stock = yf.Ticker(ticker)
#         market_cap = stock.info.get('marketCap')
#         return market_cap
#     except Exception as e:
#         print(f"Error fetching data for {ticker}: {e}")
#         return None

# # Create a list to store the tickers and their market caps
# ticker_data = []

# for ticker in tickers:
#     market_cap = get_market_cap(ticker)
#     if market_cap is not None:
#         ticker_data.append({'Ticker': ticker, 'MarketCap': market_cap})
#     else:
#         print(f"Skipping {ticker} due to error")

# # Create a DataFrame and sort by market cap
# ticker_df = pd.DataFrame(ticker_data)
# ticker_df = ticker_df.sort_values(by='MarketCap', ascending=False)

# # Select the top 200 companies by market cap
# top_200 = ticker_df.head(200)
# print(top_200['Ticker'].tolist())  # Print the list of top 200 tickers

# def convert_data_type(df, time_cols=[], float_cols=[], cat_cols=[]):
#     for col in time_cols:
#         df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
#     for col in float_cols:
#         df[col] = df[col].astype(float)
#     for col in cat_cols:
#         df[col] = df[col].astype('category')
#     return df

# # Define the date range
# start_date = "2022-11-01"
# end_date = "2024-11-03"

# # Download data for the top 200 tickers
# tickers = top_200['Ticker'].tolist()
# df = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')
# df = df.stack(level=0).reset_index()
# df.columns = ['time', 'ticker', 'open', 'high', 'low', 'close', 'adjclose', 'volume']
# df = convert_data_type(df, time_cols=['time'], float_cols=['open', 'high', 'low', 'close', 'volume'], cat_cols=['ticker'])

# # Set the time as index and sort values by time
# df = df.set_index('time', drop=True).reset_index()
# df = df.groupby('ticker').apply(lambda x: x.sort_values('time')).reset_index(drop=True)

# required_df = df[['ticker','time','close']]

# # Convert the dataframe to TimeSeriesDataFrame
# ts_df = TimeSeriesDataFrame.from_data_frame(
#     required_df,
#     id_column='ticker',
#     timestamp_column='time',
#     target_column='close'
# )

# # Split the data into train and test sets
# train_data = ts_df.slice_by_timestamp(None, "2024-09-03")
# test_data = ts_df.slice_by_timestamp("2024-09-04", None)

# # Initialize and fit the TimeSeriesPredictor
# predictor = TimeSeriesPredictor(
#     prediction_length=30,  # Predict 30 days into the future
#     target="close",
#     eval_metric="MASE"
# )

# predictor.fit(
#     train_data,
#     time_limit=3600,  # Time limit in seconds (1 hour in this case)
#     presets="medium_quality"
# )

# # Make predictions
# predictions = predictor.predict(test_data)

# # Evaluate the model
# evaluation = predictor.evaluate(test_data)
# print(f"Evaluation results: {evaluation}")

# # Get feature importance
# feature_importance = predictor.feature_importance(test_data)
# print(f"Feature importance: {feature_importance}")

# # Save the predictions to a CSV file
# predictions.to_csv('stock_predictions.csv')
# print("Predictions saved to 'stock_predictions.csv'")

ImportError: cannot import name 'get_chunk_n_rows' from 'sklearn.utils' (/usr/local/lib/python3.10/dist-packages/sklearn/utils/__init__.py)

In [ ]:
# from pmdarima import auto_arima
# from statsmodels.tsa.seasonal import seasonal_decompose
# from sklearn.metrics import mean_absolute_error, mean_squared_error
# from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
# def autogluon_dataloader(df, prediction_horizon, timestamp_col):
#     train_timeseries_df = df.iloc[:-prediction_horizon]

#     train_timeseries_df = TimeSeriesDataFrame.from_data_frame(
#         train_timeseries_df,
#         id_column="ticker",
#         timestamp_column=timestamp_col
#     )

#     test_timeseries_df = TimeSeriesDataFrame.from_data_frame(
#         df,
#         id_column="ticker",
#         timestamp_column=timestamp_col
#     )

#     return train_timeseries_df, test_timeseries_df

# def trainer(train_timeseries_df, prediction_horizon, target_col, save_path, eval_metric, quality_preset='best_quality', training_time=10):
#     predictor = TimeSeriesPredictor(
#         freq='B',
#         prediction_length=prediction_horizon,
#         path=save_path,
#         target=target_col,
#         eval_metric=eval_metric,
#     )

#     predictor.fit(
#         train_timeseries_df,
#         presets=quality_preset,
#         time_limit=60 * training_time
#     )

#     predictions = predictor.predict(train_timeseries_df)
#     return predictions, predictor

In [ ]:
# required_df.head()

In [ ]:
# def forecast(required_df,horizon):
#   required_df = add_missing_business_days(required_df)
#   required_df = upscale(required_df, 10)
#   required_df["ID"] = 1
#   train_r1_timeseries_df, test_r1_timeseries_df = autogluon_dataloader(required_df, horizon, 'PICKED_DATE_TIME')
#   r1_predictions, r1_predictor = trainer(train_r1_timeseries_df, horizon, target_col='TRANSACTION_QUANTITY',
#                                         save_path='Code\Pickup history\Autogluon Models\Autgluon-timerseries',
#                                         quality_preset='best_quality', eval_metric='MAPE', training_time=30)
#   r1_test_leaderboard = r1_predictor.leaderboard(test_r1_timeseries_df)
#   r1_train_summary = r1_predictor.fit_summary()

#   r1_best_model = r1_train_summary['model_best']
#   r1_best_model_metric = r1_train_summary['model_performance'][r1_best_model]

#   print(f"\nBest model for R1: {r1_best_model}")
#   print(f"Best R1 model {r1_predictor.eval_metric}: {r1_best_model_metric}")

#   #


In [ ]:
all_rdf = []

for i in os.listdir('/content/drive/MyDrive/processed'):
    name = i.strip("processed_news_sentiment_").strip(".csv")
    rdf = pd.read_csv(f'/content/drive/MyDrive/processed/{i}')
    rdf['ticker'] = name
    all_rdf.append(rdf)

for rdf in all_rdf:
    rdf.rename(columns={'Date': 'time'}, inplace=True)
    rdf['time'] = pd.to_datetime(rdf['time']).dt.strftime('%Y-%m-%d 00:00:00+00:00')

combined_df = pd.concat(all_rdf, ignore_index=True)

In [ ]:
combined_df['time'] = pd.to_datetime(combined_df['time'], utc=True)
combined_df

,time,Overall Sentiment Score,Summary,ticker
0,2024-10-25 00:00:00+00:00,0.207768,Investors with a lot of money to spend have ta...,XOM
1,2024-10-26 00:00:00+00:00,0.324414,This ETF could help steadily grow your wealth ...,XOM
2,2024-10-27 00:00:00+00:00,0.263680,Both tech companies are capturing customers hu...,XOM
3,2024-10-28 00:00:00+00:00,0.121578,"In the latest trading session, Exxon Mobil (XO...",XOM
4,2024-10-29 00:00:00+00:00,0.274006,As CVX gears up to report third-quarter earnin...,XOM
...,...,...,...,...
2436,2024-10-21 00:00:00+00:00,0.252671,APH's third-quarter 2024 results are likely to...,CARR
2437,2024-10-22 00:00:00+00:00,0.151146,"In the current market session, Carrier Global ...",CARR
2438,2024-10-24 00:00:00+00:00,0.075997,CARR's third-quarter 2024 results reflect stre...,CARR
2439,2024-10-25 00:00:00+00:00,0.024898,FFIV's fourth-quarter fiscal 2024 results are ...,CARR


In [ ]:
print(df['time'].dtype)
print(combined_df['time'].dtype)

datetime64[ns, UTC]
datetime64[ns, UTC]


In [ ]:
merged_df = pd.merge(df, combined_df,
                     on=['time', 'ticker'],
                     how='inner')

In [ ]:
merged_df

,time,ticker,open,high,low,close,adjclose,volume,Overall Sentiment Score,Summary
0,2024-10-30 00:00:00+00:00,AAPL,232.610001,233.470001,229.550003,230.100006,230.100006,47070900.0,0.172857,"As we've stated in this space previously, the ..."
1,2024-10-31 00:00:00+00:00,AAPL,229.339996,229.830002,225.369995,225.910004,225.910004,64370100.0,0.199834,Market uncertainty has crept back into market ...
2,2024-11-01 00:00:00+00:00,AAPL,220.970001,225.350006,220.270004,222.910004,222.910004,65242200.0,0.194196,The chip supplier really had a fiscal second q...
3,2024-10-17 00:00:00+00:00,ABBV,190.539993,191.619995,188.460007,188.570007,188.570007,4419000.0,0.200902,AbbVie (ABBV) reachead $188.57 at the closing ...
4,2024-10-18 00:00:00+00:00,ABBV,188.970001,189.690002,188.160004,188.860001,188.860001,5607900.0,0.223410,J&J tops third-quarter earnings and sales esti...
...,...,...,...,...,...,...,...,...,...,...
1967,2024-10-28 00:00:00+00:00,XOM,116.790001,119.010002,116.769997,118.900002,118.900002,11844500.0,0.121578,"In the latest trading session, Exxon Mobil (XO..."
1968,2024-10-29 00:00:00+00:00,XOM,118.580002,119.169998,117.070000,117.279999,117.279999,14731000.0,0.274006,As CVX gears up to report third-quarter earnin...
1969,2024-10-30 00:00:00+00:00,XOM,117.660004,118.269997,116.480003,116.690002,116.690002,12303700.0,0.261924,A favorable pipeline throughput volume is like...
1970,2024-10-31 00:00:00+00:00,XOM,117.150002,118.669998,116.680000,116.779999,116.779999,20512000.0,0.249852,Stocks fell broadly during Thursday's session ...


In [ ]:
df = merged_df

In [ ]:
def split_dataset(df):
    df = df.sort_values(['ticker', 'time'])
    df['time_idx'] = df.groupby('ticker').cumcount()

    # Assuming 30 days per month and 7 days per week
    total_days = df.groupby('ticker').size().max()
    test_days = 30  # One week for test
    val_days = 30  # One month for validation
    train_days = total_days - val_days - test_days

    training_data = df[df['time_idx'] < train_days]
    validation_data = df[(df['time_idx'] >= train_days) & (df['time_idx'] < train_days + val_days)]
    test_data = df[df['time_idx'] >= train_days + val_days]

    return training_data, validation_data, test_data

# Apply the function to your DataFrame
training_df, validation_df, test_df = split_dataset(df)

In [ ]:
def verify_split(training_df, validation_df, test_df, expected_val_test_entries=30):
    train_counts = training_df.groupby('ticker').size()
    val_counts = validation_df.groupby('ticker').size()
    test_counts = test_df.groupby('ticker').size()

    print("Sample of training set ticker counts:")
    print(train_counts.head())
    print("\nSample of validation set ticker counts:")
    print(val_counts.head())
    print("\nSample of test set ticker counts:")
    print(test_counts.head())

    all_tickers = set(train_counts.index) | set(val_counts.index) | set(test_counts.index)

    validation_issues = []
    test_issues = []

    for ticker in all_tickers:
        if ticker in val_counts and val_counts[ticker] != expected_val_test_entries:
            validation_issues.append(f"Ticker {ticker}: {val_counts[ticker]} entries (expected {expected_val_test_entries})")
        if ticker in test_counts and test_counts[ticker] != expected_val_test_entries:
            test_issues.append(f"Ticker {ticker}: {test_counts[ticker]} entries (expected {expected_val_test_entries})")

    if validation_issues:
        print("\nValidation set issues:")
        for issue in validation_issues[:5]:  # Print first 5 issues
            print(issue)
        print(f"... and {len(validation_issues) - 5} more issues") if len(validation_issues) > 5 else None

    if test_issues:
        print("\nTest set issues:")
        for issue in test_issues[:5]:  # Print first 5 issues
            print(issue)
        print(f"... and {len(test_issues) - 5} more issues") if len(test_issues) > 5 else None

    if not validation_issues and not test_issues:
        print(f"\nVerification passed: All tickers have exactly {expected_val_test_entries} entries in both validation and test sets.")
    else:
        print(f"\nVerification failed: Some tickers do not have {expected_val_test_entries} entries in validation or test sets.")

    print(f"\nTotal tickers: {len(all_tickers)}")
    print(f"Tickers in training set: {len(train_counts)}")
    print(f"Tickers in validation set: {len(val_counts)}")
    print(f"Tickers in test set: {len(test_counts)}")

# Verify the split datasets
verify_split(training_df, validation_df, test_df)

Sample of training set ticker counts:
ticker
AAPL      3
ABBV     11
AMZN      2
AVGO      9
BX      164
dtype: int64

Sample of validation set ticker counts:
ticker
CARR    22
CME     30
COF     20
MAR     30
MCD     30
dtype: int64

Sample of test set ticker counts:
ticker
CME    14
MAR     6
MCD    30
dtype: int64

Validation set issues:
Ticker COF: 20 entries (expected 30)
Ticker CARR: 22 entries (expected 30)

Test set issues:
Ticker CME: 14 entries (expected 30)
Ticker MAR: 6 entries (expected 30)

Verification failed: Some tickers do not have 30 entries in validation or test sets.

Total tickers: 26
Tickers in training set: 26
Tickers in validation set: 5
Tickers in test set: 3


In [ ]:
print(validation_df.shape)  # Check the shape of validation data
print(validation_df.head())  # Print the first few rows of validation data
print(validation_df['ticker'].value_counts())  # Check how many entries per ticker


(132, 11)
                         time ticker       open       high        low  \
417 2024-09-16 00:00:00+00:00   CARR  77.080002  77.459999  76.129997   
418 2024-09-17 00:00:00+00:00   CARR  77.500000  78.000000  76.680000   
419 2024-09-24 00:00:00+00:00   CARR  81.000000  81.644997  80.160004   
420 2024-09-25 00:00:00+00:00   CARR  80.470001  80.834999  79.209999   
421 2024-09-26 00:00:00+00:00   CARR  80.419998  80.521004  79.410004   

         close   adjclose     volume  Overall Sentiment Score  \
417  77.110001  76.909027  3970500.0                 0.287281   
418  77.959999  77.756813  3517100.0                 0.303624   
419  80.410004  80.200432  3520400.0                 0.355479   
420  79.669998  79.462349  3589200.0                 0.334020   
421  79.779999  79.572067  3123700.0                 0.207264   

                                               Summary  time_idx  
417                   Style Box ETF report for VOE ...       228  
418  The AI revolution is 

In [ ]:
max_prediction_length = 3  # One week
max_encoder_length = 5  # One month, adjust if needed

training_dataset = TimeSeriesDataSet(
    training_df,
    time_idx="time_idx",
    target="close",
    group_ids=["ticker"],
    min_encoder_length=max_encoder_length // 2,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["ticker"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["close"],
    target_normalizer=GroupNormalizer(groups=["ticker"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

# Create validation and test datasets
validation_dataset = TimeSeriesDataSet.from_dataset(training_dataset, validation_df, stop_randomization=True)
test_dataset = TimeSeriesDataSet.from_dataset(training_dataset, test_df, stop_randomization=True)

/usr/local/lib/python3.10/dist-packages/pytorch_forecasting/data/timeseries.py:1282: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 3 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__ticker': 'AMZN'}, {'__group_id__ticker': 'NVDA'}, {'__group_id__ticker': 'TSLA'}]
  warnings.warn(


In [ ]:
batch_size = 128  # Increased batch size for faster training
train_dataloader = training_dataset.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation_dataset.to_dataloader(train=False, batch_size=batch_size * 2, num_workers=0)
test_dataloader = test_dataset.to_dataloader(train=False, batch_size=batch_size * 2, num_workers=0)

In [ ]:
pl.seed_everything(42)

tft = TemporalFusionTransformer.from_dataset(
    training_dataset,
    learning_rate=0.001,  # Reduced learning rate for better stability
    hidden_size=64,  # Increased hidden size for more capacity
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=32,
    loss=QuantileLoss(),
    optimizer="adamw",  # Changed to AdamW optimizer
    reduce_on_plateau_patience=4,
)

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()
logger = TensorBoardLogger("lightning_logs")

In [ ]:
import torch

print("Is CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

Is CUDA available: True
Current device: Tesla T4


In [ ]:
trainer = pl.Trainer(
    max_epochs=100,
    accelerator="auto",  # Colab will automatically detect and use GPU if available
    devices=1,
    enable_model_summary=True,
    gradient_clip_val=0.1,
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
    log_every_n_steps=50,
    val_check_interval=None,
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader)


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 260    | train
3  | prescalers                         | ModuleDict                      | 384    | train
4  | static_variable_selection          | VariableSelectionNetwork        | 20.9 K | train
5  | encoder_variable_selection         | VariableSelectionNetwork        | 20.7 K | train
6  | decoder_variable_selection         | VariableSelectionNetwork        | 13.7 K | train
7  | static_context_variable_selectio

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (14) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
tickers

['XOM',
 'UNH',
 'AAPL',
 'LLY',
 'NVDA',
 'AVGO',
 'ABBV',
 'TSLA',
 'MA',
 'JNJ',
 'PG',
 'HD',
 'AMZN',
 'COST',
 'NFLX',
 'WMT',
 'V',
 'ORCL',
 'MCD',
 'COF',
 'CMG',
 'CAT',
 'BX',
 'MAR',
 'CME',
 'CARR']

In [ ]:
# Print the first batch to see its structure
for batch in test_dataloader:
    print("Batch structure:", type(batch), "Length:", len(batch))
    for item in batch:
        print("Item type:", type(item), "Shape:", item.shape if hasattr(item, 'shape') else "No shape")
    break  # Just print the first batch

Batch structure: <class 'tuple'> Length: 2
Item type: <class 'dict'> Shape: No shape
Item type: <class 'tuple'> Shape: No shape


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
from pytorch_forecasting.metrics import MAE

# Perform prediction
predictions = tft.predict(test_dataloader, return_x=True)

# Determine the device
device = next(tft.parameters()).device

# Move predictions to CPU (since we'll convert to numpy later)
predicted_values = predictions.output.cpu().numpy()

# Calculate MAE for the test set
actuals = torch.cat([y[0].cpu() for x, y in iter(test_dataloader)]).numpy() # Move actuals to CPU as well

predicted_values = predicted_values.ravel()
actuals = actuals.ravel()

# Check for NaN values
nan_mask = np.isnan(predicted_values) | np.isnan(actuals)

if np.any(nan_mask):
    print("NaN values found:")
    print(f"Total NaN values: {np.sum(nan_mask)}")

    # Remove NaN values
    valid_indices = ~nan_mask
    predicted_values = predicted_values[valid_indices]
    actuals = actuals[valid_indices]

# Calculate MAE
mae_value = mean_absolute_error(actuals, predicted_values)

# Calculate MSE
mse = mean_squared_error(actuals, predicted_values)

print(f"Test MAE: {mae_value}")
print(f"Test MSE: {mse}")

# Additional information
print(f"Number of samples after removing NaNs: {len(predicted_values)}")
print(f"Predictions range: {np.min(predicted_values)} to {np.max(predicted_values)}")
print(f"Actuals range: {np.min(actuals)} to {np.max(actuals)}")

# Diagnostic information
print("\nDiagnostic Information:")
print(f"Shape of predicted_values: {predicted_values.shape}")
print(f"Shape of actuals: {actuals.shape}")
print(f"Type of predicted_values: {type(predicted_values)}")
print(f"Type of actuals: {type(actuals)}")



INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


NaN values found:
Total NaN values: 18
Test MAE: 6.357134819030762
Test MSE: 57.8490104675293
Number of samples after removing NaNs: 153
Predictions range: 216.26974487304688 to 303.23590087890625
Actuals range: 223.61000061035156 to 314.8900146484375

Diagnostic Information:
Shape of predicted_values: (153,)
Shape of actuals: (153,)
Type of predicted_values: <class 'numpy.ndarray'>
Type of actuals: <class 'numpy.ndarray'>
